In [1]:
import random
import asyncio
import websockets
import numpy as np

from task_first import get_numbers

### generate random sequence

In [2]:
def generate_random_list(t):
    """Generate list of random digits of the length t
    """
    return [random.randint(0,9) for i in range(t)]

In [3]:
t = 21
random_int_list = generate_random_list(t)
# distribution of digits in random_int_list
histogram = [1/t for i in range(t)]
random_int_list

[3, 4, 2, 6, 6, 5, 5, 7, 5, 7, 3, 2, 5, 0, 0, 0, 9, 6, 2, 5, 6]

### generating ethalon images

In [4]:
# default width = 3, height = 5
width_scale = 40
height_scale = 20
# list, at which on a position i, array which represents i
int_to_image_list = asyncio.get_event_loop().run_until_complete(get_numbers(width_scale,height_scale))
# random_int_list - list of random digits, e.g. [4,5,6,8]
ethalon_matrices_list = [int_to_image_list[random_int] for random_int in random_int_list]
print(ethalon_matrices_list[0].shape)

(100, 120)


### noising images

In [5]:
w, h = 5*height_scale,3*width_scale
print(w,h)
p = 0.9
noised_images = list()
for matrix in ethalon_matrices_list:
    noise = np.random.choice(a=[0, 1], size=(w,h), p=[p, 1-p])
    noised_image = np.bitwise_xor(matrix,noise)
    noised_images.append(noised_image)

100 120


In [11]:
def calculate_prob(x_noised,ethalon_matrices,target_index,histogram,p_):
    """Due to the seminar notation, this fuction try to calculate p(x_noised,true_matrices_list[target_index])
    Args:
        x_noised - ndarray 100*120
        true_matrices_list - the list of 10 ndarray 100*120
        target_index - int in [0,..,9]
        p - float in [0,1]
    
    """
    xor_target = x_noised^ethalon_matrices[target_index]
    result_sum = 0
    for index_matrix,true_matrix in enumerate(ethalon_matrices):
        xor_matrix = x_noised^true_matrix
        # the equation from the seminar
        result_sum += histogram[index_matrix]*(1-p_/p_)**((xor_matrix - xor_target).sum())
        #print(result_sum)
    #print((1-p_/p_))
    return histogram[target_index]/result_sum

###### check prediction

In [23]:
target_index = 2
prob_max = 0
index = 0
for i in range(10):
    current_prob = calculate_prob(noised_images[target_index],int_to_image_list,i,histogram,p)
    if current_prob > prob_max:
        index = i
print("predicted digit",index)
print("real digit :",random_int_list[target_index])

predicted digit 2
real digit : 2


c:\install\python\python36\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in power
  from ipykernel import kernelapp as app


### recursive type

In [17]:
def calc_prob_list(digit_index):
    """Calculate a list of possibilities that digit_index corresponds to [0,..,9] 
    """
    possible_sum = list()
    for i in range(10):
        prob = calculate_prob(noised_images[digit_index],int_to_image_list,i,histogram,p)
        possible_sum.append(prob)
    return possible_sum
calc_prob_list(0)

c:\install\python\python36\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in power
  from ipykernel import kernelapp as app


[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [18]:
def calc_partial_sum(previous_probs_list_,current_probs_list_,t):
    """This function is convolution when dimension = 2.
    Args:
        previous_probs_list - a list of length = 9*t +1,where at position i, the probability, that the
            first (t-1) digits' sum = i
        current_probs_list_ -  a list of length = 10, ,where at position i, the probability, that the
            t-th digit equals to i
        t - int, denotes current position , we calculate sums for
    """
    predictions = list(np.zeros((9*(t+1) +1)))
    for current_value,current_prob in enumerate(current_probs_list_):
        for prev_value,prev_prob in enumerate(previous_probs_list_):
            predictions[current_value + prev_value] += current_prob*prev_prob
            #print(current_prob)
    return predictions

In [21]:
t = 2
for current_t in range(0,t):
    if current_t == 0 : 
        previous_probs_list = calc_prob_list(0)
    else:
        next_probs_list = calc_prob_list(current_t)
        previous_probs_list = calc_partial_sum(previous_probs_list,next_probs_list,current_t)

c:\install\python\python36\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in power
  from ipykernel import kernelapp as app


In [22]:
print("predicted sum ",np.array(previous_probs_list).argmax())
print("real sum ",np.sum(np.array(random_int_list[:t])).sum())

predicted sum  7
real sum  7
